# Task

In [11]:
#Import Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [12]:
#Function to fetch election results
def fetch_election_results(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve data from {url}")
        return None

In [13]:
#Function to parse election results
def parse_election_results(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    # Attempt to find the main table containing election results
    main_table = soup.find('table', class_='table-party')
    
    if main_table:
        # Initialize lists to store data
        constituencies = []
        candidates = []
        parties = []
        votes = []
        leads = []

        # Extract data from each row in the table
        rows = main_table.find_all('tr')
        for row in rows[1:]:  # skip header row
            cols = row.find_all('td')
            if len(cols) == 7:  # ensure it's a data row
                constituency = cols[0].text.strip()
                candidate = cols[1].text.strip()
                party = cols[2].text.strip()
                vote = int(cols[3].text.strip().replace(',', ''))
                lead = int(cols[4].text.strip().replace(',', ''))

                constituencies.append(constituency)
                candidates.append(candidate)
                parties.append(party)
                votes.append(vote)
                leads.append(lead)

        # Create a DataFrame
        data = pd.DataFrame({
            'Constituency': constituencies,
            'Candidate': candidates,
            'Party': parties,
            'Votes': votes,
            'Lead': leads
        })

        return data
    else:
        print("Main table 'table-party' not found.")
        return pd.DataFrame()

In [14]:
#Function to derive election results
def derive_insights(data):
    insights = {}
    
    # Insight 1: Total number of constituencies
    insights['Total Constituencies'] = len(data['Constituency'].unique())
    
    # Insight 2: Total number of candidates
    insights['Total Candidates'] = len(data['Candidate'].unique())
    
    # Insight 3: Party with the maximum seats
    party_with_max_seats = data.loc[data.groupby('Party')['Votes'].idxmax(), 'Party'].iloc[0]
    insights['Party with Maximum Seats'] = party_with_max_seats
    
    # Insight 4: Constituency with the highest voter turnout
    highest_turnout_constituency = data.loc[data['Votes'].idxmax(), 'Constituency']
    insights['Constituency with Highest Voter Turnout'] = highest_turnout_constituency
    
    # Insight 5: Total votes polled across all constituencies
    total_votes_polled = data['Votes'].sum()
    insights['Total Votes Polled'] = total_votes_polled
    
    # Insight 6: Average lead margin across all constituencies
    average_lead_margin = data['Lead'].mean()
    insights['Average Lead Margin'] = average_lead_margin
    
    # Insight 7: Top 5 constituencies with the highest number of votes
    top_constituencies_by_votes = data.nlargest(5, 'Votes')[['Constituency', 'Votes']]
    insights['Top 5 Constituencies by Votes'] = top_constituencies_by_votes
    
    # Insight 8: Number of uncontested constituencies
    uncontested_constituencies = data[data['Votes'] == 0]['Constituency'].count()
    insights['Number of Uncontested Constituencies'] = uncontested_constituencies
    
    # Insight 9: Party-wise distribution of seats
    party_wise_seat_distribution = data.groupby('Party')['Votes'].sum().nlargest(5)
    insights['Top 5 Parties by Seats'] = party_wise_seat_distribution
    
    # Insight 10: Percentage of seats won by the leading party
    total_seats = data['Votes'].count()
    seats_won_by_max_party = data[data['Party'] == party_with_max_seats]['Votes'].count()
    insights['Percentage of Seats Won by Leading Party'] = (seats_won_by_max_party / total_seats) * 100
    
    return insights

In [15]:
# Main Execution
url = "https://results.eci.gov.in"

# Fetching and parsing data
html = fetch_election_results(url)
if html:
    election_data = parse_election_results(html)
    if not election_data.empty:
        insights = derive_insights(election_data)
        # Display insights
        for key, value in insights.items():
            print(f"{key}: {value}")
    else:
        print("No election data found.")
else:
    print("Failed to fetch election data.")